#**Problem Description:**
>Significant investments are being made to improve building efficiencies to reduce costs and emissions. The question is, are the improvements working? Under pay-for-performance financing, *the building owner makes payments based on the difference between their real energy consumption and what they would have used without any retrofits.* The latter values have to come from a model. Current methods of estimation are fragmented and do not scale well. Some assume a specific meter type or don’t work with different building types.




#**Solution Description:**
> We have to develop accurate models of metered building energy usage in the following areas:  
1. Chilled water
2. Electric
3. Hot water
4. Steam meters

> The data comes from over 1,000 buildings over a three-year timeframe. With better estimates of these energy-saving investments, large scale investors and financial institutions will be more inclined to invest in this area to enable progress in building efficiencies.









In [2]:
'''
Direcotry: /content/drive/My Drive/Csc-219-Project/DataSet

building_metadata.csv
sample_submission.csv
test.csv
train.csv
weather_test.csv
weather_train.csv

'''

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Importing all the libraries in this block
import pandas as pd
import pandas_profiling as pdp
%matplotlib inline
import missingno as msno

import seaborn as sb
import matplotlib.pyplot as plt
import missingno as ms
import numpy as np


import os,random, math, psutil, pickle


#libraries for Data Preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import lightgbm as lgb
from sklearn.model_selection import train_test_split

#library for cost fuction
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



**Phase-1: Data Collection**

All raw data that we gonna use in this project.

In [4]:
%%time
raw_building_meta = pd.read_csv("/content/drive/My Drive/Csc-219-Project/DataSet/building_metadata.csv")
raw_sample_data = pd.read_csv("/content/drive/My Drive/Csc-219-Project/DataSet/sample_submission.csv")
raw_test = pd.read_csv("/content/drive/My Drive/Csc-219-Project/DataSet/test.csv")
raw_train = pd.read_csv("/content/drive/My Drive/Csc-219-Project/DataSet/train.csv")
raw_weather_test = pd.read_csv("/content/drive/My Drive/Csc-219-Project/DataSet/weather_test.csv")
raw_weather_train = pd.read_csv("/content/drive/My Drive/Csc-219-Project/DataSet/weather_train.csv")




CPU times: user 23.5 s, sys: 4.24 s, total: 27.7 s
Wall time: 1min 40s


In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df



def plot_dist_col(column):
  import seaborn as sns
  '''plot dist curves for train and test weather data for the given column name'''
  fig, ax = plt.subplots(figsize=(10, 10))
  sns.distplot(raw_weather_train[column].dropna(), color='green', ax=ax).set_title(column, fontsize=16)
  sns.distplot(raw_weather_test[column].dropna(), color='purple', ax=ax).set_title(column, fontsize=16)
  plt.xlabel(column, fontsize=15)
  plt.legend(['train', 'test'])
  plt.show()



def report_missing_data(df):
    print('Total Number of rows :', len(df))
    for column in df.columns:
        print(column,':\t\t', 'Missing rows:\t', sum(df[column].isnull()), '|\t\t', '% Missing:\t {:.2f}'.format(sum(df[column].isnull())*100/len(df)),'%')


Reduce the size of data as its current size is >1Gb.

In [6]:
%%time 
## Reducing memory of the data frames
train = reduce_mem_usage(raw_train)
test = reduce_mem_usage(raw_test)
weather_train = reduce_mem_usage(raw_weather_train)
weather_test = reduce_mem_usage(raw_weather_test)
building_metadata = reduce_mem_usage(raw_building_meta)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)
CPU times: user 1.26 s, sys: 194 ms, total: 1.46 s
Wall time: 1.46 s


**Phase-1a: Univariate Analysis**

*Most of the data visualisation are done through Pandas Profiling which has show in the report.html.*

To detect outliers we are plotting normal distribution chart for each feature.

We removed outliers problem via using Standard Scaling all the features in the training data.

No data preprocessing is done on the test data!

In [7]:
'''#plotting normal distribution 
plot_dist_col('air_temperature')
plot_dist_col('cloud_coverage')
plot_dist_col('dew_temperature')
plot_dist_col('precip_depth_1_hr')
plot_dist_col('sea_level_pressure')
plot_dist_col('wind_speed')'''

"#plotting normal distribution \nplot_dist_col('air_temperature')\nplot_dist_col('cloud_coverage')\nplot_dist_col('dew_temperature')\nplot_dist_col('precip_depth_1_hr')\nplot_dist_col('sea_level_pressure')\nplot_dist_col('wind_speed')"

In [8]:
# First merge train and building data
train = pd.merge(train,building_metadata,how = 'left')           
print(train.shape)
train.head()

# Then Merge train_building with weather_train data
train = pd.merge(train,weather_train, on = ['site_id','timestamp'], how = 'left')
print(train.shape)

# First merge test and building data
test = pd.merge(test,building_metadata,how = 'left')           
print(test.shape)
test.head()

# Now Merge test_building with weather_test data
test = pd.merge(test,weather_test, on = ['site_id','timestamp'], how = 'left')
print(test.shape)
test.head()


(20216100, 9)
(20216100, 16)
(41697600, 9)
(41697600, 16)


,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,2008.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,2004.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,1991.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,2002.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,1975.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609


In [9]:
'''
#to convert data into pickle

train_df.to_pickle('train_df.pkl')
test_df.to_pickle('test_df.pkl')
   
del train_df, test_df
gc.collect()
'''

'''
#read pickle
train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')

'''

#%%time 
#report = pdp.ProfileReport(train)

"\n#read pickle\ntrain_df = pd.read_pickle('train_df.pkl')\ntest_df = pd.read_pickle('test_df.pkl')\n\n"

In [10]:
report_missing_data(train)

Total Number of rows : 20216100
building_id :		 Missing rows:	 0 |		 % Missing:	 0.00 %
meter :		 Missing rows:	 0 |		 % Missing:	 0.00 %
timestamp :		 Missing rows:	 0 |		 % Missing:	 0.00 %
meter_reading :		 Missing rows:	 0 |		 % Missing:	 0.00 %
site_id :		 Missing rows:	 0 |		 % Missing:	 0.00 %
primary_use :		 Missing rows:	 0 |		 % Missing:	 0.00 %
square_feet :		 Missing rows:	 0 |		 % Missing:	 0.00 %
year_built :		 Missing rows:	 12127645 |		 % Missing:	 59.99 %
floor_count :		 Missing rows:	 16709167 |		 % Missing:	 82.65 %
air_temperature :		 Missing rows:	 96658 |		 % Missing:	 0.48 %
cloud_coverage :		 Missing rows:	 8825365 |		 % Missing:	 43.66 %
dew_temperature :		 Missing rows:	 100140 |		 % Missing:	 0.50 %
precip_depth_1_hr :		 Missing rows:	 3749023 |		 % Missing:	 18.54 %
sea_level_pressure :		 Missing rows:	 1231669 |		 % Missing:	 6.09 %
wind_direction :		 Missing rows:	 1449048 |		 % Missing:	 7.17 %
wind_speed :		 Missing rows:	 143676 |		 % Missing:	 0.71 %


**Phase-2: Data Cleansing**

There are 6 tables and all of them contains null values. First task is to impute all null values so that after that we can atleast plot our graphs.


In [79]:
'''
#msno.matrix(train)
from google.colab import files
'''
pdp.ProfileReport(train.iloc[:20000])


Number of variables,16
Number of observations,20000
Total Missing (%),0.0%
Total size in memory,1.6 MiB
Average record size in memory,83.0 B
Numeric,15
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,1


**Handling categorical features**

In [0]:
'''#dendo=msno.dendrogram(train)

matrix = msno.matrix(train.sample(200))'''
le = LabelEncoder()

# train_df['primary_use'] = train_df['primary_use'].astype(str)
train['primary_use'] = le.fit_transform(train['primary_use']).astype(np.int8)

# test_df['primary_use'] = test_df['primary_use'].astype(str)
test['primary_use'] = le.fit_transform(test['primary_use']).astype(np.int8)

**Feature engineering**

Deriving age from year built.

In [0]:
train['age'] = train['year_built'].max() - train['year_built'] + 1
test['age'] = test['year_built'].max() - test['year_built'] + 1

**Handling missing values**

To streamline this though process it is useful to know the 3 categories in which missing data can be classified into:
1.   Missing Completely at Random (MCAR)
2.   Missing at Random (MAR)
3.   Missing Not at Random (MNAR)



In [0]:
train['floor_count'] = train['floor_count'].fillna(-999).astype(np.int16)
test['floor_count'] = test['floor_count'].fillna(-999).astype(np.int16)

train['year_built'] = train['year_built'].fillna(-999).astype(np.int16)
test['year_built'] = test['year_built'].fillna(-999).astype(np.int16)

#train['age'] = train['age'].fillna(-999).astype(np.int16)
#test['age'] = test['age'].fillna(-999).astype(np.int16)

train['cloud_coverage'] = train['cloud_coverage'].fillna(-999).astype(np.int16)
test['cloud_coverage'] = test['cloud_coverage'].fillna(-999).astype(np.int16) 

**Handling missing data with imputer(For train)**

In [0]:
import numpy as np
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [27]:

imputer.fit(train[["air_temperature"]])
train['air_temperature'] = imputer.transform(train[["air_temperature"]])
print("missing values in precip_depth_1_hr(after imputer):",train.precip_depth_1_hr.isnull().sum())

print("missing values in Air_temperature(before imputer):",train.air_temperature.isnull().sum())
imputer.fit(train[["precip_depth_1_hr"]])
train['precip_depth_1_hr'] = imputer.transform(train[["precip_depth_1_hr"]])
print("missing values in precip_depth_1_hr(after imputer):",train.precip_depth_1_hr.isnull().sum())


print("missing values in dew_temperature(before imputer):",train.dew_temperature.isnull().sum())
imputer.fit(train[["dew_temperature"]])
train['dew_temperature'] = imputer.transform(train[["dew_temperature"]])
print("missing values in dew_temperature(after imputer):",train.dew_temperature.isnull().sum())


print("missing values in wind_direction(before imputer):",train.wind_direction.isnull().sum())
imputer.fit(train[["wind_direction"]])
train['wind_direction'] = imputer.transform(train[["wind_direction"]])
print("missing values in wind_direction(after imputer):",train.wind_direction.isnull().sum())



print("missing values in sea_level_pressure(before imputer):",train.sea_level_pressure.isnull().sum())
imputer.fit(train[["sea_level_pressure"]])
train['sea_level_pressure'] = imputer.transform(train[["sea_level_pressure"]])
print("missing values in sea_level_pressure(after imputer):",train.sea_level_pressure.isnull().sum())


print("missing values in wind_speed(before imputer):",train.wind_speed.isnull().sum())
imputer.fit(train[["wind_speed"]])
train['wind_speed'] = imputer.transform(train[["wind_speed"]])
print("missing values in wind_speed(after imputer):",train.wind_speed.isnull().sum())

print("missing values in age(before imputer):",train.age.isnull().sum())
imputer.fit(train[["age"]])
train['age'] = imputer.transform(train[["age"]])
print("missing values in age(after imputer):",train.age.isnull().sum())

missing values in precip_depth_1_hr(after imputer): 186681
missing values in Air_temperature(before imputer): 0
missing values in precip_depth_1_hr(after imputer): 0
missing values in dew_temperature(before imputer): 22190
missing values in dew_temperature(after imputer): 0
missing values in wind_direction(before imputer): 65486
missing values in wind_direction(after imputer): 0
missing values in sea_level_pressure(before imputer): 108073
missing values in sea_level_pressure(after imputer): 0
missing values in wind_speed(before imputer): 23378
missing values in wind_speed(after imputer): 0
missing values in age(before imputer): 596820
missing values in age(after imputer): 0


In [29]:
train.isnull().any()

building_id           False
meter                 False
meter_reading         False
site_id               False
primary_use           False
square_feet           False
year_built            False
floor_count           False
air_temperature       False
cloud_coverage        False
dew_temperature       False
precip_depth_1_hr     False
sea_level_pressure    False
wind_direction        False
wind_speed            False
age                   False
dtype: bool

Removing timestamp for the model.


In [0]:
train = train.drop(columns=['timestamp'])

Phase-3: Splitting test and train data

Train = 85%
Test  = 15%

In [0]:
from sklearn.model_selection import train_test_split

#train = train.iloc[0:1000000,]
train_set, test_set = train_test_split(train, test_size=0.15)


target = 'meter'
features = list(train.columns)
features = [f for f in features if f!=target]


X_tr = train_set[features]
y_tr = train_set[[target]]

X_te = test_set[features]
y_te = test_set[[target]]



Scaling the training data

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr = scaler.fit_transform(X_tr)
X_te = scaler.transform(X_te)

In [0]:
#pdp.ProfileReport(train.iloc[:20000])

In [0]:
#cost function
def rmse(predicted, test):
  lin_reg_mse = mean_squared_error(test, predicted)
  lin_reg_rmse = np.sqrt(lin_reg_mse)
  print(lin_reg_rmse)

Phase-4: Model selection



> We choosed 4 models:
1. Linear Regression
2. Ridge
3. Light Gradient Boosting Model(for regression)
4. Neural Networks regressor.




In [36]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from time import time

lin_reg = LinearRegression()
start = time()
lin_reg.fit(X_tr, np.ravel(y_tr))
print("lin_reg took %.2f seconds for candidates"
      " parameter settings." % (time() - start))

pred_train= lin_reg.predict(X_tr)
pred_test = lin_reg.predict(X_te)

lin_reg took 0.28 seconds for candidates parameter settings.


Training and test error for detecting overfitting or underfitting.

In [40]:
print("Train rmse:")
train_rmse = rmse(pred_train, y_tr)
print("\nTest rmse:")
test_rmse = rmse(pred_test, y_te)  

Train rmse:
0.8716823561554352

Test rmse:
0.8706234603590223


Ridge Model

In [41]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


param_grid = [{'alpha': [0.600,0.69,0.699,0.7, 0.705, 0.777,0.799, 0.8, 0.889,0.899, 0.9,]}]
scoring_method = 'r2'

grid_search_rr = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error', verbose= 2)
start = time()
grid_search_rr.fit(X_tr, y_tr)
print("grid_search_rr took %.2f seconds"
      " parameter settings." % (time() - start))

Fitting 3 folds for each of 11 candidates, totalling 33 fits
[CV] alpha=0.6 .......................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ........................................ alpha=0.6, total=   0.2s
[CV] alpha=0.6 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] ........................................ alpha=0.6, total=   0.2s
[CV] alpha=0.6 .......................................................
[CV] ........................................ alpha=0.6, total=   0.2s
[CV] alpha=0.69 ......................................................
[CV] ....................................... alpha=0.69, total=   0.2s
[CV] alpha=0.69 ......................................................
[CV] ....................................... alpha=0.69, total=   0.2s
[CV] alpha=0.69 ......................................................
[CV] ....................................... alpha=0.69, total=   0.2s
[CV] alpha=0.699 .....................................................
[CV] ...................................... alpha=0.699, total=   0.2s
[CV] alpha=0.699 .....................................................
[CV] ...................................... alpha=0.699, total=   0.2s
[CV] alpha=0.699 .....................................................
[CV] .

[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    7.0s finished


Training and test error for detecting overfitting or underfitting.

In [46]:
print("Best Params: ")
print(grid_search_rr.best_params_)
print("sqrt: ")
print(np.sqrt(-grid_search_rr.best_score_))
print("Best Score: ")
print(grid_search_rr.best_score_)

rr_model = grid_search_rr.best_estimator_
pred_ridge = rr_model.predict(X_te)

print("\n**Test rmse:")
rmse(pred_ridge, y_te)

pred_ridge = rr_model.predict(X_tr)
print("\n**Train rmse:")
rmse(pred_ridge, y_tr)

Best Params: 
{'alpha': 0.9}
sqrt: 
0.8717186788912231
Best Score: 
-0.7598934551278593

**Test rmse:
0.870623431363034

**Train rmse:
0.871682356236957


In [0]:
lgb_train = lgb.Dataset(X_tr, y_tr)
lgb_val = lgb.Dataset(X_te, y_te)

import lightgbm as lgb


params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmsle',
    'max_depth': 8,
    'num_leaves': 77, 
    'learning_rate': 0.09,
    'min_samples_split' : 700,
    'verbose': 10,
    'subsample' : 0.8,
    'n_estimators' : 160,
    'random_state': 8,
    'max_features': 7,
    'num_iterations' :3500
    }


lgbm_model = lgb.train(params, train_set = lgb_train, valid_sets = lgb_val, verbose_eval=1)




In [0]:
test_pred_lgb = lgbm_model.predict(X_te)
train_pred_lgb = lgbm_model.predict(X_tr)

In [178]:
print("\n**Train rmse:")
rmse(train_pred_lgb, y_tr)

print("\n**Test rmse:")
test_rmse_lgb=rmse(test_pred_lgb, y_te)


**Train rmse:
0.24226924729675892

**Test rmse:
0.29156041425987694


Neural Network Model: MLPRegressor 

In [0]:
from sklearn.neural_network import MLPRegressor
MLP_reg = MLPRegressor(activation='relu',max_iter = 15, verbose = 'true', hidden_layer_sizes =(5,), solver='adam',
                                     learning_rate='adaptive',)


In [0]:
param_grid = {'alpha': [0.001,0.005,0.0075, 0.008, 0.009]
              }
grid_search_MLPR = GridSearchCV(MLP_reg, param_grid,  cv=5, scoring='neg_mean_squared_error', verbose = 2)

In [61]:
start = time()
grid_search_MLPR.fit(X_tr, y_tr)
print("grid_search_MLPR took %.2f seconds"
      " parameter settings." % (time() - start))

grid_search

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] alpha=0.001 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Iteration 1, loss = 0.38873593
Iteration 2, loss = 0.34305503
Iteration 3, loss = 0.32364677
Iteration 4, loss = 0.31374757
Iteration 5, loss = 0.30907719
Iteration 6, loss = 0.30540030
Iteration 7, loss = 0.30297993
Iteration 8, loss = 0.30094266
Iteration 9, loss = 0.29743212
Iteration 10, loss = 0.29541430
Iteration 11, loss = 0.29444251
Iteration 12, loss = 0.29329471
Iteration 13, loss = 0.29244844
Iteration 14, loss = 0.29205906
Iteration 15, loss = 0.29187237
[CV] ...................................... alpha=0.001, total=  12.8s
[CV] alpha=0.001 .....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.8s remaining:    0.0s


Iteration 1, loss = 0.41091447
Iteration 2, loss = 0.35993593
Iteration 3, loss = 0.33916680
Iteration 4, loss = 0.32338423
Iteration 5, loss = 0.31340696
Iteration 6, loss = 0.30596808
Iteration 7, loss = 0.30170923
Iteration 8, loss = 0.29918307
Iteration 9, loss = 0.29775014
Iteration 10, loss = 0.29666650
Iteration 11, loss = 0.29595815
Iteration 12, loss = 0.29533380
Iteration 13, loss = 0.29503780
Iteration 14, loss = 0.29467755
Iteration 15, loss = 0.29445425
[CV] ...................................... alpha=0.001, total=  12.6s
[CV] alpha=0.001 .....................................................
Iteration 1, loss = 0.40585525
Iteration 2, loss = 0.35627555
Iteration 3, loss = 0.33034366
Iteration 4, loss = 0.31657454
Iteration 5, loss = 0.30913075
Iteration 6, loss = 0.30557666
Iteration 7, loss = 0.30363233
Iteration 8, loss = 0.30256899
Iteration 9, loss = 0.30213105
Iteration 10, loss = 0.30171580
Iteration 11, loss = 0.30156410
Iteration 12, loss = 0.30136618
Iteration 13

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  4.2min finished


Iteration 1, loss = 0.38974316
Iteration 2, loss = 0.34661094
Iteration 3, loss = 0.32434640
Iteration 4, loss = 0.31347257
Iteration 5, loss = 0.30760827
Iteration 6, loss = 0.30420281
Iteration 7, loss = 0.30012482
Iteration 8, loss = 0.29596266
Iteration 9, loss = 0.29329561
Iteration 10, loss = 0.29127001
Iteration 11, loss = 0.29040038
Iteration 12, loss = 0.28991588
Iteration 13, loss = 0.28954651
Iteration 14, loss = 0.28929117
Iteration 15, loss = 0.28890891
grid_search_MLPR took 270.47 seconds parameter settings.


In [195]:
#model=grid_search_MLPR.best_estimator_
model = MLPRegressor(activation='relu', alpha=0.05, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=50, learning_rate='adaptive',
             learning_rate_init=0.001, max_iter=100, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=1000, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.15, verbose='true', warm_start=False)

model.fit(X_tr,y_tr)

Iteration 1, loss = 0.35332349
Iteration 2, loss = 0.32150534
Iteration 3, loss = 0.30796254
Iteration 4, loss = 0.30155866
Iteration 5, loss = 0.29796846
Iteration 6, loss = 0.29603591
Iteration 7, loss = 0.29463402
Iteration 8, loss = 0.29397760
Iteration 9, loss = 0.29286618
Iteration 10, loss = 0.29266735
Iteration 11, loss = 0.29193542
Iteration 12, loss = 0.29180998
Iteration 13, loss = 0.29140392
Iteration 14, loss = 0.29109353
Iteration 15, loss = 0.29081943
Iteration 16, loss = 0.29055331
Iteration 17, loss = 0.29031810
Iteration 18, loss = 0.29039018
Iteration 19, loss = 0.29019613
Iteration 20, loss = 0.29006317
Iteration 21, loss = 0.28971013
Iteration 22, loss = 0.28966246
Iteration 23, loss = 0.28934025
Iteration 24, loss = 0.28938069
Iteration 25, loss = 0.28925457
Iteration 26, loss = 0.28927816
Iteration 27, loss = 0.28944324
Iteration 28, loss = 0.28872742
Iteration 29, loss = 0.28875200
Iteration 30, loss = 0.28861450
Iteration 31, loss = 0.28868305
Iteration 32, los

MLPRegressor(activation='relu', alpha=0.05, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=50, learning_rate='adaptive',
             learning_rate_init=0.001, max_iter=100, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=1000, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.15, verbose='true', warm_start=False)

In [196]:
pred_mlrp = model.predict(X_tr)
print("\n**Train rmse:")
rmse(pred_mlrp, y_tr)

pred_mlrp = model.predict(X_te)

print("\n**Test rmse:")
rmse(pred_mlrp, y_te)


**Train rmse:
0.7329422470347852

**Test rmse:
0.7324916614472208
